# Übung 1 Aufgabe 4 - Implementierung von Optimierungsmodellen

In [ ]:
!pip install pulp

Importieren Sie das Python-Modul von PuLP.

In [1]:
from pulp import *

## Produktionsprogrammplanung

Im Folgenden sind die Modellparameter als Dictionaries dargestellt:

In [2]:
#Indexmenge der Produkte:
produkte = [1,2]

#Verkaufspreise von Produkt 1 und 2
preise = {
    1 : 3,
    2 : 4
}

#Maschinenstunden von Produkt 1 und 2
maschinenstunden = {
    1 : 3,
    2 : 2
}

#Rohstoffverbrauch von Produkt 1 und 2
rohstoffverbrauch = {
    1 : 5,
    2 : 10
}

#Arbeitsstunden von Produkt 1 und 2
arbeitsstunden = {
    1 : 0,
    2 : 0.5
}

#Ressourcenverfügbarkeiten
ressourcen = {
    "maschine": 1200,
    "rohstoffe": 3000,
    "arbeitskraefte": 125
}

Nutzen Sie die <code>LpProblem()</code>-Funktion, um ein Modell zu initialisieren:

In [3]:
prob = LpProblem("Produktionsprogrammplanung", LpMaximize)

Befüllen Sie die ein Dictionary <code>variables</code>mit den Entscheidungsvariablen für die Produkte 1 und 2 mit der LpVariable()-Funktion:

In [4]:
variables = {}
for produkt in produkte:
    variables[produkt] = LpVariable("x_"+str(produkt), 0, cat=LpContinuous)

Definieren Sie die Zielfunktion, welche den Gewinn maximiert:

In [5]:
prob += (
    lpSum(preise[produkt] * variables[produkt] for produkt in produkte), "Gewinnfunktion"
)

Implementieren Sie die Nebenbedingungen, welche die Kapazitätsbeschränkungen der Ressourcen abbilden:

In [6]:
#Kapazität Maschinenstunden:
prob += (
    lpSum(variables[produkt] * maschinenstunden[produkt] for produkt in produkte) <= ressourcen["maschine"], "Maschinenkapazitaet"
)

#Kapazität Rohstoffe:
prob += (
    lpSum(variables[produkt] * rohstoffverbrauch[produkt] for produkt in produkte) <= ressourcen["rohstoffe"], "Rohstoffkapazitaet"
)

#Kapazität Arbeitsstunden:
prob += (
    lpSum(variables[produkt] * arbeitsstunden[produkt] for produkt in produkte) <= ressourcen["arbeitskraefte"], "Arbeitskapazitaet"
)

Lösen Sie das Modell und lassen Sie sich die Laufzeit, den optimalen Zielfunktionswert und die optimalen Variablenwerte ausgeben:

In [ ]:
#Lösen
prob.solve()

#Logging
if(prob.status == 1):
    print(f"Das Modell wurde optimal geloest. Der optimale Zielfunktionswert betraegt {value(prob.objective)}.")
    print(f"Die Loesungszeit betrug {value(prob.solutionTime)} Sekunden.")
    print()
    print(f"Optimales Prduktionsprogramm:")
    for produkt in produkte:
        print(f"Produktionsmenge von Produkt {produkt}: {variables[produkt].varValue}")

Führen Sie nach Lösen des Modells den folgenden Codeblock aus, um Ihre Lösung zu überprüfen:

In [ ]:
#Überprüfungscodeblock
allCorrect = True
if(prob.status != 1):
    print("Das Modell wurde nicht optimal gelöst.")
    allCorrect = False
if(value(prob.objective) != 1500):
    print("Der Zielfunktionswert ist nicht korrekt.")
    allCorrect = False
if(variables[1].varValue != 300):
    print("Die Produktionsmenge von Produkt 1 ist nicht korrekt.")
    allCorrect = False
if(variables[2].varValue != 150):
    print("Die Produktionsmenge von Produkt 2 ist nicht korrekt.")
    allCorrect = False
if(allCorrect):
    print("Das Modell scheint richtig implementiert zu sein.")

## Lerngruppenplanung 

Implementieren Sie das Modell zur Lerngruppenplanung in der Version aus Teilaufgabe e). Benennen Sie Ihre Modellvariable <code>prob</code> und fügen Sie die besuchten Lerngruppen der Liste <code>besuchte_gruppen</code> hinzu. Anschließend können Sie den Überprüfungscodeblock ausführen.

In [ ]:
besuchte_gruppen = []
#Parameter

lerngruppen=[1,2,3]
nutzenfaktor = {
    1: 2,
    2: 1,
    3: 0.5
}
zeitaufwand = {
    1: 30,
    2: 20,
    3: 3
}
zeitbudget = 40

#Modell definieren

prob = LpProblem("Lerngruppenplanung", LpMaximize)

#Variablen:
y={}
for g in lerngruppen:
    y[g] = LpVariable("y_"+str(g), cat=LpBinary)
x = LpVariable("x", cat=LpBinary)

#Zielfunktion
prob += (
    lpSum(nutzenfaktor[g] * y[g] for g in lerngruppen) + 0.5 * x, "Nutzenfunktion"
)

#Nebenbedingungen
prob += (
    lpSum(zeitaufwand[g] * y[g] for g in lerngruppen) <= zeitbudget, "Zeitkapazitaet"
)

prob += y[1] + y[3] <= 1, "Gruppe 1 oder 3"

prob += x <= 0.5 * (y[2] + y[3]), "Synergie Gruppen 2 und 3"

#Lösen
prob.solve()

for g in lerngruppen:
    if y[g].varValue == 1:
        besuchte_gruppen.append(g)
        
#Logging
print(f"Folgende Lerngruppen werden besucht: {besuchte_gruppen}")

In [ ]:
#Überprüfungscodeblock
besuchte_gruppen = set(besuchte_gruppen)
allCorrect = True
if(prob.status != 1):
    print("Das Modell wurde nicht optimal gelöst.")
    allCorrect = False
if(value(prob.objective) != 2):
    print("Der Zielfunktionswert ist nicht korrekt.")
    allCorrect = False
if(2 in besuchte_gruppen == False or 3 in besuchte_gruppen == False or 1 in besuchte_gruppen):
    print("Die ermittelte Lerngruppenentscheidung ist nicht korrekt.")
    allCorrect = False
if(allCorrect):
    print("Das Modell scheint richtig implementiert zu sein.")